<ul class="breadcrumb">
  <li><a href="1.1.Univaraite_Linear_Mixed_Effects_Model.ipynb">Univariate Basics</a></li>
  <li><a href="2.1.Univariate_Linear_Additive_Genetic_Model.ipynb">Univariate Additive Genetic Model</a></li> 
  <li><a href="3.1.Univariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Univariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univaraite Linear Mixed Effects Model with Genomic Data</font> 
</div>


<button type="button" class="btn btn-lg btn-primary">Data</button>

In [ ]:
using DataFrames,CSV,JWAS,JWAS.Datasets

In [ ]:
phenofile = Datasets.dataset("testMME","simple.txt")
genofile  = Datasets.dataset("testMME","genotype.txt")
pedfile   = Datasets.dataset("testMME","pedigree.txt");

In [ ]:
;cat $phenofile

In [ ]:
;cat $genofile

In [ ]:
;cat $pedfile

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [ ]:
phenotype = CSV.read(phenofile,delim = ',',header=true,null="NA");

In [ ]:
pedigree = get_pedigree(pedfile);

In [ ]:
residual_variance = 1.0
genetic_variance  = 2.5
genetic_variance_by_marker    = 1.5
genetic_variance_by_polygenic = 1.0;

In [ ]:
model = build_model("y = intercept + Age + Animal",residual_variance)
set_covariate(model,"Age")
set_random(model,"Animal",pedigree,genetic_variance_by_polygenic)

In [ ]:
add_markers(model,genofile,genetic_variance_by_marker,separator=',');

<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [ ]:
output=runMCMC(model,phenotype,chain_length=5000,
methods="BayesC",Pi=0.9,estimatePi=true,output_samples_frequency=100);

In [ ]:
keys(output)

<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [ ]:
output["Posterior mean of location parameters"]

In [ ]:
output["Posterior mean of marker effects"]

In [ ]:
using Plots
histogram(output["MCMC samples for: π"])